In [ ]:
import mysql.connector
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import plotly.io as pio
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import ta
db = mysql.connector.connect(
  host="localhost",
  user="root",
  password="devshree090103",
  database="stock_price"
)
cursor = db.cursor()
cursor.execute("SELECT DISTINCT script FROM dailyprice")
scripts = [row[0] for row in cursor.fetchall()]
cursor.close()
choice=1
while(choice==1):
    choice=int(input('''Welcome to StockWise!!
                    Enter 1 to Continue Further
                    Enter and any number to exit
                    '''))
    if choice==1:
        script = input(f"Enter a script to predict ({', '.join(scripts)}): ")
        if script not in scripts:
            print(f"Error: script {script} not found")
        cursor = db.cursor()
        cursor.execute(f"SELECT Date, Script, Open, High, Low, Close, Volume FROM dailyprice WHERE script='{script}' ORDER BY date")
        rows = cursor.fetchall()
        cursor.close()
        
        df = pd.DataFrame(rows, columns=['Date', 'Script','Open','High','Low','Close','Volume'])
        df['7 EH']=df['High'].ewm(span=7).mean()
        df['7 LH']=df['Low'].ewm(span=7).mean()
        df['High_divisional']=(df['High'].ewm(span=7).mean()+df['High'].ewm(span=14).mean()+df['High'].ewm(span=21).mean())/3
        df['Low_divisional']=(df['Low'].ewm(span=7).mean()+df['Low'].ewm(span=14).mean()+df['Low'].ewm(span=21).mean())/3
        sma = df['Close'].rolling(window=7).mean()
        std = df['Close'].rolling(window=7).std()
        df['Bollinger Upper'] = sma + 2*std
        df['Bollinger Middle'] = sma
        df['Bollinger Lower'] = sma - 2*std
        rsi = ta.momentum.RSIIndicator(df['Close'], window=7)
        df['RSI'] = rsi.rsi()
        df['Stock_occ'] = ta.momentum.stoch(df['High'], df['Low'], df['Close'], window=7)
        df['Stock_sig'] = df['Stock_occ'].rolling(window=3).mean()
        df['MACD'] = ta.trend.macd(df['Close'], window_slow=3, window_fast=1)
        df['Signal line'] = ta.trend.macd_signal(df['Close'], window_slow=3, window_fast=3, window_sign=3)
        print(df.tail(1))
        
        df = df.fillna(df.mean())
        X = df[['High_divisional', 'Bollinger Middle', 'RSI', 'Stock_occ','Stock_sig', 'MACD','Signal line']]
        y = df['Close']
        # split the data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
        # train the regression model
        reg_model = LinearRegression()
        reg_model.fit(X_train, y_train)
        # train the classification model
        clf_model = RandomForestClassifier(n_estimators=100)
        clf_model.fit(X_train, y_train.apply(lambda x: 1 if x > y_train.mean() else 0))
        # make a prediction for tomorrow's price using the regression model
        tomorrow_features = X.tail(1)
        tomorrow_price = reg_model.predict(tomorrow_features)[0]
        print(f"Tomorrow's estimated price: {tomorrow_price}")
        # make a prediction for whether to buy or sell using the classification model
        buy_or_sell = clf_model.predict(tomorrow_features)[0]
        if buy_or_sell == 1:
            print("Recommendation: Buy")
        else:
            print("Recommendation: Sell")
        #evaluate the accuracy of the classification model
        y_pred = clf_model.predict(X_test)
        accuracy = accuracy_score(y_test.apply(lambda x: 1 if x > y_train.mean() else 0), y_pred)
        print(f"Accuracy of buy/sell classification: {accuracy}")
        
        #Define candlestick trace
        candlestick = go.Candlestick(x=df['Date'],open=df['Open'],high=df['High'],low=df['Low'],close=df['Close'],
                                     name='OHLC',increasing=dict(line=dict(color='green')),decreasing=dict(line=dict(color='red')))
        #Define line plots
        seven_eh = go.Scatter(x=df['Date'],y=df['7 EH'],name='7 EH')
        seven_lh = go.Scatter(x=df['Date'],y=df['7 LH'],name='7 LH')
        #Define scatter plots
        bollinger_upper = go.Scatter(x=df['Date'],y=df['Bollinger Upper'],name='Bollinger Upper')
        bollinger_middle = go.Scatter(x=df['Date'],y=df['Bollinger Middle'],name='Bollinger Middle')
        bollinger_lower = go.Scatter(x=df['Date'],y=df['Bollinger Lower'],name='Bollinger Lower')
        #Define histogram trace
        volume = go.Bar(x=df['Date'],y=df['Volume'],name='Volume',
                        marker=dict(color=df['Close'].diff().fillna(0).apply(lambda x: 'red' if x < 0 else 'green')))
        histogram = go.Histogram(x=df['Volume'],name='Volume Distribution',marker=dict(color='gray'),yaxis='y2')
        #Define layout
        layout = go.Layout(title='Interactive Graphs',yaxis=dict(title='Price'),yaxis2=dict(title='Volume', 
                           overlaying='y',side='right'),xaxis=dict(rangeslider=dict(visible=False)),showlegend=True)
        #Define figure
        fig = go.Figure(data=[candlestick, volume, histogram], layout=layout)
        #Define figure
        fig = go.Figure(data=[candlestick, seven_eh, seven_lh, bollinger_upper, bollinger_middle, bollinger_lower, volume],
                        layout=layout)
        buttons = [{'label': 'None', 'method': 'update', 'args': [{'visible': [False, False, False, False, False, False, False]},
                   {'title': 'Please select any one button to view'}]},
                   
                   {'label': 'Prices', 'method': 'update', 'args': [{'visible': [True, False, False, False, False, False, False]},
                   {'title': 'Prices'}]},
                   
                   {'label': 'Divisional', 'method': 'update', 'args': [{'visible': [False, True, True, False, False, False, 
                    False]}, {'title': 'Divisional'}]},
                   
                   {'label': 'Bollinger Band', 'method': 'update', 'args': [{'visible': [False, False, False, True, True, True, 
                    False]}, {'title': 'Bollinger Band'}]},
                   
                   {'label': 'Volume', 'method': 'update', 'args': [{'visible':[False, False, False, False, False, False, True]}, 
                   {'title': 'Volume'}]}]
        # Add dropdown menu to layout
        layout.update({'updatemenus': [{'type': 'dropdown','buttons': buttons}]})
        # Set the default visibility of the traces
        fig.update_layout(updatemenus=[
                        dict(type="buttons",buttons=[dict(label="None",method="update",
                        args=[{"visible":[False,False,False,False,False,False,False]},
                        {"title":"Please select any one button to view"}]),
                                                                      
                        dict(label="Prices",method="update",args=[{"visible": [True, False, False, False, False, False, False]},
                        {"title": "Prices"}]),
                                                                      
                        dict(label="Divisional",method="update",args=[{"visible": [False, True, True, False, False, False, False]},
                         {"title": "Divisional"}]),
                                                                      
                        dict(label="Bollinger Band",method="update",args=[{"visible": [False, False, False, True, True, True, 
                        False]},{"title": "Bollinger Band"}]),
                                                                      
                       dict(label="Volume",method="update",args=[{"visible": [False, False, False, False, False, False, True]},
                        {"title": "Volume"}])])])
        #Show the figure
        pio.show(fig)
    else:
        print('''Thank you for visiting StockWise
                 Have a nice day''')
        break

Welcome to StockWise!!
                    Enter 1 to Continue Further
                    Enter and any number to exit
                    1
Enter a script to predict (bharti, infy, reliance, tatasteel, tcs): bharti
           Date  Script   Open    High    Low  Close  Volume        7 EH  \
295  2023-03-10  bharti  773.0  778.95  768.0  773.7  146953  775.469161   

           7 LH  High_divisional  Low_divisional  Bollinger Upper  \
295  760.326107       775.343403      760.140753       787.224731   

     Bollinger Middle  Bollinger Lower        RSI  Stock_occ  Stock_sig  \
295        762.214286        737.20384  61.221851  63.237774  67.737987   

         MACD  Signal line  
295  2.491513          0.0  


C:\Users\VITESH SHAH\AppData\Local\Temp\ipykernel_49528\2517431343.py:54: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df = df.fillna(df.mean())


Tomorrow's estimated price: 776.4377761162167
Recommendation: Buy
Accuracy of buy/sell classification: 0.9887640449438202
